### Imports

In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import seaborn
import os
import glob

import pandas as pd

### Functions

In [2]:
def find_nearest(array,value):
    idx,val = min(enumerate(array), key=lambda x: abs(x[1]-value))
    return idx

### Define paths and read data

In [3]:
# Paths and ensemble members
EXP="HIST2"
OP="/work/scratch-pw3/atb299/"#+EXP+"/"
ENS=sorted(os.listdir(OP))

### Open extracted AMOC variables - small files for ease of use

In [4]:
EM_amoc_26N = xr.open_dataset(OP+"Ensemble_amoc_z_26N.nc").squeeze()
EM_amoc_42N = xr.open_dataset(OP+"Ensemble_amoc_z_42N.nc").squeeze()
EM_amoc_50N = xr.open_dataset(OP+"Ensemble_amoc_z_50N.nc").squeeze()
EM_amoc_55N = xr.open_dataset(OP+"Ensemble_amoc_z_55N.nc").squeeze()

In [5]:
z1k = find_nearest(EM_amoc_26N['depthw'],1000)
EM_amoc_26N['zomsfatl_1k'] = EM_amoc_26N['zomsfatl'].isel(depthw=z1k)
EM_amoc_42N['zomsfatl_1k'] = EM_amoc_42N['zomsfatl'].isel(depthw=z1k)
EM_amoc_50N['zomsfatl_1k'] = EM_amoc_50N['zomsfatl'].isel(depthw=z1k)
EM_amoc_55N['zomsfatl_1k'] = EM_amoc_55N['zomsfatl'].isel(depthw=z1k)

In [6]:
# Annual, annual mean and variance
EM_amoc_26N_annual = EM_amoc_26N.resample(time_counter='1Y').mean(dim='time_counter')
EM_amoc_26N_annual['zomsfatl_1k_mean'] = EM_amoc_26N_annual['zomsfatl_1k'].mean(dim='ensemble_member') 
EM_amoc_26N_annual['zomsfatl_1k_var'] = EM_amoc_26N_annual['zomsfatl_1k'].var(dim='ensemble_member')

# Monthly mean and variance
EM_amoc_26N['zomsfatl_1k_mean'] = EM_amoc_26N['zomsfatl_1k'].mean(dim='ensemble_member')
EM_amoc_26N['zomsfatl_1k_var'] = EM_amoc_26N['zomsfatl_1k'].var(dim='ensemble_member')

# Monthly seasonal cycle
EM_amoc_26N['zomsfatl_1k_mean_scyc'] = EM_amoc_26N['zomsfatl_1k_mean'].groupby('time_counter.month').mean("time_counter")
EM_amoc_26N['zomsfatl_1k_varmean_scyc'] = EM_amoc_26N['zomsfatl_1k_mean'].groupby('time_counter.month').var("time_counter")

EM_amoc_26N['zomsfatl_1k_ensvar_as_scyc'] = EM_amoc_26N['zomsfatl_1k_var'].groupby('time_counter.month').mean("time_counter")


# Ensemble with mean seasonal cycle removed
EM_amoc_26N['zomsfatl_1k_anom'] = EM_amoc_26N['zomsfatl_1k'].groupby('time_counter.month') - EM_amoc_26N['zomsfatl_1k_mean_scyc'] # Remove mean seasonal cycle
EM_amoc_26N['zomsfatl_1k_mean_anom'] = EM_amoc_26N['zomsfatl_1k_mean'].groupby('time_counter.month') - EM_amoc_26N['zomsfatl_1k_mean_scyc'] # Remove mean seasonal cycle

In [7]:
# RAPID Obs
Ramoc = xr.open_dataset('/work/scratch-pw3/atb299/RAPID_moc_transports.nc')
Ramoc_M = Ramoc.resample(time='1M').mean() # Create monthly means - date assigned is last day of the month
offset = pd.tseries.frequencies.to_offset("15D") # Use pandas to offset the assigned time to (apprx.) centre of the month
Ramoc_M["time"] = Ramoc_M.get_index("time") - offset # ...
Ramoc_M['AMOC26_1k_sc'] = Ramoc_M['moc_mar_hc10'].groupby('time.month').mean("time")
Ramoc_M['AMOC26_1k_sc_var'] = Ramoc_M['moc_mar_hc10'].groupby('time.month').var("time")
Ramoc_M['AMOC26_1k_sc11on'] = Ramoc_M['moc_mar_hc10'].sel(time=slice('2011-01-01', '2022-12-31')).groupby('time.month').mean("time")
Ramoc_M['AMOC26_1k_scpre10'] = Ramoc_M['moc_mar_hc10'].sel(time=slice('2004-04-01', '2009-12-31')).groupby('time.month').mean("time")

Ramoc_Y = Ramoc.resample(time='1Y').mean() # Create annual means
offset = pd.tseries.frequencies.to_offset("183D") # Use pandas to offset the assigned time to (apprx.) centre of the month
Ramoc_Y["time"] = Ramoc_Y.get_index("time") - offset # ...

# Convert to datetime64 format        
Ramoc_M360 = Ramoc_M.convert_calendar(calendar = '360_day', align_on = 'date')
Ramoc_Y360 = Ramoc_Y.convert_calendar(calendar = '360_day', align_on = 'date')

### Plot AMOC time series

In [8]:
fp = "/home/users/atb299/CANARI/Figures/"

In [ ]:
EM_amoc_26N['zomsfatl_1k'][:,:].plot.line(hue='ensemble_member',size=6,aspect=2,add_legend=True);
EM_amoc_26N['zomsfatl_1k_mean'].plot(lw=3,color="white");
EM_amoc_26N['zomsfatl_1k_mean'].plot(lw=1,color="gray");
Ramoc_M360['moc_mar_hc10'][:].plot.line(label="RAPID",color='k')
seaborn.move_legend(plt.gca(), loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("AMOC @ 26N, 1 km: monthly time series for all ensemble members")
plt.savefig(fp+'AMOC26N_1k_monthly_ts.png', dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
EM_amoc_26N_annual['zomsfatl_1k'].plot.line(hue='ensemble_member',size=6,aspect=2);
EM_amoc_26N_annual['zomsfatl_1k_mean'].plot(lw=5,color="white");
EM_amoc_26N_annual['zomsfatl_1k_mean'].plot(lw=2,color="gray");
Ramoc_Y360['moc_mar_hc10'][:-1].plot.line(label="RAPID",color='k')
seaborn.move_legend(plt.gca(), loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("AMOC @ 26N, 1 km: calendar annual time series for all ensemble members")
plt.savefig(fp+'AMOC26N_1k_annual_ts.png', dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
EM_amoc_26N

In [ ]:
EM_amoc_26N['zomsfatl_1k_anom'].plot.line(hue='ensemble_member',size=6,aspect=2);
EM_amoc_26N['zomsfatl_1k_mean_anom'].plot(lw=3,color="white")
EM_amoc_26N['zomsfatl_1k_mean_anom'].plot(lw=1,color="gray")
seaborn.move_legend(plt.gca(), loc='center left', bbox_to_anchor=(1, 0.5));
plt.title("AMOC @ 26N, 1 km: monthly time series for all ensemble members with mean scyc removed");
plt.savefig(fp+'AMOC26N_1k_monthly_ts_minus_mean_scyc.png', dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
EM_amoc_26N['zomsfatl_1k_var'].plot(size=6,aspect=2);
plt.title("AMOC @ 26N, 1 km: monthly ensemble variance");
plt.savefig(fp+'AMOC26N_1k_monthly_ensemble_variance.png', dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
EM_amoc_26N['zomsfatl_1k_ensvar_as_scyc'].plot(size=6,aspect=2)
plt.title("AMOC @ 26N, 1 km: mean scyc of ensemble variance");
plt.savefig(fp+'AMOC26N_1k_mean_scyc_of_monthly_ensemble_variance.png', dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

In [ ]:
EM_amoc_26N['zomsfatl_1k_mean_scyc'].plot(size=6,aspect=2,label="LE mean scyc");
Ramoc_M['AMOC26_1k_sc'].plot(label="RAPID");
slo = (EM_amoc_26N['zomsfatl_1k_mean_scyc']-EM_amoc_26N['zomsfatl_1k_ensvar_as_scyc']**.5).squeeze()
shi = (EM_amoc_26N['zomsfatl_1k_mean_scyc']+EM_amoc_26N['zomsfatl_1k_ensvar_as_scyc']**.5).squeeze()

rlo = (Ramoc_M['AMOC26_1k_sc']-Ramoc_M['AMOC26_1k_sc_var']**.5).squeeze()
rhi = (Ramoc_M['AMOC26_1k_sc']+Ramoc_M['AMOC26_1k_sc_var']**.5).squeeze()
plt.legend();
plt.title("AMOC @ 26N, 1 km: Mean seasonal cycle (Sv)");

plt.fill_between(EM_amoc_26N['month'],slo,shi,alpha=.4);
#plt.fill_between(EM_amoc_26N['month'],rlo,rhi,alpha=.3);
plt.savefig(fp+'AMOC26N_1k_mean_scyc_vs_RAPID.png', dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)